In [1]:
using VariPEPS
using TensorKit
using OptimKit


In this notebook we show how to import your own model to be used in the `VariPEPS.jl` code. This will allow you to calculate expectation values as well as the gradient for variational optimization.\
For a basic introduction into the functions used in this notebook, first take a look at the `basic_functionality.ipynb` notebook.

Specify the Bond-dimension `d`, the environment-bond-dimension `χ`. 

In [2]:
χ = 8
d = 2;

Here specify the unit cell. For this we define an array "Pattern_arr" that represents the unit cell structure.\
The structure is [x-coodinate, y-coordinate].\
If two positions within the Pattern_arr are filled with the same number, we assume the tensors at these positions to be identical.\
The choice below corresponds to a unit cell like:   
A   B \
B   A

In [3]:
Pattern_arr = Array{Any}(undef,2,2)

Pattern_arr[1,1] = 1
Pattern_arr[2,1] = 2
Pattern_arr[1,2] = 2
Pattern_arr[2,2] = 1;

As an example we wish to use the J1J2-model. This model is already implemented in the code but we pretend here that we need to import it.

Importing a model works via the keyword **model**. We can pass an array of functions that are then used for the calculation of the energy expectation value and for the calculation of observables of interest. 
Ham_parameters = ?



Regarding the other keywords:\
**lattice**: tells us which lattice we use.\
**Projector_type**: During the CTMRG algorithm we need projectors. These can be choosen in different ways. Here we choose the :half option. An other more expensive choice would be :fullfishman.\
**Space_type**: Here we use ℂ.



Now the user needs to create functions that create all the terms in the Hamiltonian that we want.\
The function `create_spin_onehalf_operators(; Space_type = ℂ)` just creates the local spin operators.\

The function `use_J1J2model_square_lattice(Ham_parameters, loc, pattern_arr; Space_type = ℂ)` takes several inputs:
1. The Hamiltonian parameters that we have in the model in question. Here we have `J1` & `J2` which are the couplings of neighbors and next-neighbors.\
Further we have `h` and `dir` which can be specified in case we want to include an external magnetic field in a certain direction. The Hamiltonian parameters need to be passed in a `named tuple`- see below.
2. We pass the local PEPS tensors. This is done in order that the model can automatically adjust itself to dimension of the local Hilbert space. Here we don't need it explicitly but for some other models (e.g. Bosons where one might want to limit the number of bosons per site) this is useful.
3. The Pattern array of the PEPS ansatz we are making. This is used such that we could choose a Hamiltonian that differs on every site in our unit cell. Here we do not have such a case.\

Now after defining all the terms we want to use we pass them into an arrays. As we have multiple terms for every distinct tensor in our unit cell we again pass them together as a `named tuple`. For a local term we give the name `loc`. For the horizontal exchange term we have the name `hor`, for the vertiacal one `vert`, for the diagonal one in the up-right direction we have `diag_ur` while we have `diag_dr` for the diagonal term in the down-right direction. As mentioned above we create an array that gets contains such a named tuple for every distict tensor in the unit cell of our PEPS ansatz, such that they could in priniple be different.

The function `observables_Heisenberg_model_square(ham_parameters, loc, pattern_arr; Space_type = ℂ)` creates some operators of which we want to get the expectation value for our state.


In [4]:
function create_spin_onehalf_operators(; Space_type = ℂ)
    Dim_loc = 2
    space_loc = Space_type^Dim_loc
    
    σ_p_mat =  [0 1 ; 0 0]
    σ_m_mat =  [0 0 ; 1 0]
    σ_z_mat =  0.5 * [1 0 ; 0 -1]
    σ_y_mat =  0.5* [0 -1im ; 1im 0]
    σ_x_mat =  0.5* [0 1 ; 1 0]
    
    σ_p = TensorMap(σ_p_mat, space_loc ← space_loc)
    σ_m = TensorMap(σ_m_mat, space_loc ← space_loc)
    σ_z = TensorMap(σ_z_mat, space_loc ← space_loc)
    σ_y = TensorMap(σ_y_mat, space_loc ← space_loc)
    σ_x = TensorMap(σ_x_mat, space_loc ← space_loc)
    

    return σ_x, σ_y, σ_z, σ_p, σ_m
end

function use_J1J2model_square_lattice(Ham_parameters, loc, pattern_arr; Space_type = ℂ)

    h = Ham_parameters.h
    J1 = Ham_parameters.J1
    J2 = Ham_parameters.J2
    dir = Ham_parameters.dir

    σ_x, σ_y, σ_z, σ_p, σ_m = create_spin_onehalf_operators(; Space_type = Space_type)

    #in this way the tensor is real valued.
    @tensor int_term[(i, j);(k, l)] :=  (σ_p[i,k] * σ_m[j, l] + σ_m[i,k] * σ_p[j,l])/2 + (σ_z[i,k] * σ_z[j,l])

    #add a local magnetic field if wanted
    loc_term = h * (dir[1] *  σ_x + dir[2] * σ_y + dir[3] * σ_z)

    vertical_term = J1 * int_term

    horizontal_term = J1 * int_term

    diag_ur_term = J2 * int_term

    diag_dr_term = J2 * int_term

    ham_term_arr = []

    for i in minimum(pattern_arr):maximum(pattern_arr)
        push!(ham_term_arr, (loc = loc_term, hor = horizontal_term, vert = vertical_term, diag_ur = diag_ur_term, diag_dr = diag_dr_term))
    end

    return ham_term_arr
end

function observables_Heisenberg_model_square(ham_parameters, loc, pattern_arr; Space_type = ℂ)

    #here we build the creation an annihilation operators for the truncated local Hilbert spoace of the Bosons
    σ_x, σ_y, σ_z, σ_p, σ_m = create_spin_onehalf_operators(; Space_type = Space_type);

    #=
    here we create an array of the local terms that we want to evaluate for every site in the unit cell.
    =#

    local_obs = [σ_x, σ_y, σ_z]
                         
    local_obs_arr = []

    for i in minimum(pattern_arr):maximum(pattern_arr)
        push!(local_obs_arr, (loc = local_obs))
    end

    return local_obs_arr
end

observables_Heisenberg_model_square (generic function with 1 method)

As illustrated below we just pass the two functions as elements in an array for the keyword **model**.

In [5]:
keywords = (lattice = :square, model = [use_J1J2model_square_lattice, observables_Heisenberg_model_square], Ham_parameters = (J1 = 1, J2 = 0.1, h = 0 , dir = [0,0,0]), Projector_type = :half, Space_type = ℂ)

(lattice = :square, model = Function[use_J1J2model_square_lattice, observables_Heisenberg_model_square], Ham_parameters = (J1 = 1, J2 = 0.1, h = 0, dir = [0, 0, 0]), Projector_type = :half, Space_type = ℂ)

We initialize some PEPS

In [6]:
loc_in = initialize_PEPS(d, 2, 2; seed = 1234, lattice = :square, Number_type = Float64, Space_type = ℂ); 

We can perform the ctmrg-algorithm. If we put `observ = true` we are getting the obersables we specified in the function `observables_Heisenberg_model_square()` above.

In [7]:
ctmrg_res = ctmrg(loc_in, χ, Pattern_arr; keywords..., observ = true, conv_info = false)

(0.05503212233854634, Any[-0.09524823091670581 + 8.99290180336417e-20im, -3.613908504600495e-18 + 3.857567655684017e-17im, -0.07208749549113916 + 6.31833571631044e-20im, -0.06682629773073238 + 4.29383991935067e-20im, 1.2728166167830716e-18 - 2.812288837594631e-17im, -0.0938684714159903 - 1.4810548251617073e-20im])

the function energy_and_gradient is needed for the variational optimization. It calculates the gradient at the fixed point of the CTM-RG iteration.\
It takes the same basic arguments as the function `ctmrg()`.\
It returns the the energy of the state as well as the gradient.

In [8]:
e, gr = energy_and_gradient(loc_in, χ, Pattern_arr; keywords...)

Info: it took 17 CTMRG steps to converge the SV to 1.0e-6
Info: it took 20 CTMRG steps to converge the environments element wise to 1e-6


(0.05503212131596593, TrivialTensorMap{ComplexSpace, 2, 3, Matrix{ComplexF64}}[TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2 ⊗ (ℂ^2)')):
[:, :, 1, 1, 1] =
 0.022092388434419328 + 3.7430091939084497e-17im  …  -0.031810890875525334 - 2.0159031647367335e-17im
 0.003847168088645914 - 4.195594476493727e-17im       -0.03689075400097619 + 1.2602700088835234e-16im

[:, :, 2, 1, 1] =
 0.005985911856981742 - 8.0825857045982e-17im    …  0.0031911906225355535 - 3.362662242062536e-17im
 0.014130083399787906 - 3.067551678044346e-17im     -0.007886607780129349 + 7.845653153473242e-17im

[:, :, 1, 2, 1] =
  -0.012832979399945061 - 3.072231547042756e-17im  …   -0.02779826323412252 - 1.3750480956583178e-16im
 -0.0073605045010415515 + 8.68432710206598e-17im      -0.023201728323446063 - 5.1995991521752334e-17im

[:, :, 2, 2, 1] =
  0.002692020003974966 + 2.4803340791666508e-17im  …  -0.012217745988709615 - 8.660244967290019e-17im
 -0.009480838380660602 + 4.482512836780202e-17im      0.0068759666156520014 + 1.4237557